In [1]:
import plotly.express as px
import pandas as pd
import os

In [21]:
# Carregar os dados
df_aqe = pd.read_csv('/home/guilhermerc/Documentos/workspace/pyspark-playground/output/q3/aqe/aqe.grouped')
df_wo = pd.read_csv('/home/guilhermerc/Documentos/workspace/pyspark-playground/output/q3/wo/wo.grouped')
df_cbo = pd.read_csv('/home/guilhermerc/Documentos/workspace/pyspark-playground/output/q3/cbo/cbo.grouped')
df_cboaqe = pd.read_csv('/home/guilhermerc/Documentos/workspace/pyspark-playground/output/q3/cboaqe/cboaqe.grouped')

# Adicionar uma coluna para identificar o tipo de execução
df_aqe['execution_type'] = 'Com AQE'
df_wo['execution_type'] = 'Sem AQE'
df_cbo['execution_type'] = 'Com CBO'
df_cboaqe['execution_type'] = 'Com CBO+AQE'

# Combinar os dataframes
df_combined = pd.concat([df_aqe, df_wo, df_cbo,df_cboaqe])

In [22]:
# Importar as métricas relevantes
metricas_relevantes = [
    # Métricas de tempo de execução
    "elapsedTime",  # Tempo total de execução em ms
    "executorRunTime",  # Tempo total gasto pelos executores
    "executorCpuTime",  # Tempo de CPU utilizado pelos executores
    "jvmGCTime",  # Tempo gasto em coleta de lixo
    # Eficiência de recursos e gargalos
    "shuffleFetchWaitTime",  # Tempo esperando por dados de shuffle
    "shuffleWriteTime",  # Tempo gasto escrevendo dados de shuffle
    # Utilização de memória e spill
    "peakExecutionMemory",  # Pico de memória utilizada
    "diskBytesSpilled",  # Dados despejados em disco quando memória insuficiente
    "memoryBytesSpilled",  # Dados despejados em memória
    # Métricas de shuffle
    "shuffleTotalBytesRead",  # Bytes totais lidos durante shuffle
    "shuffleBytesWritten",  # Bytes escritos durante shuffle
    "shuffleRecordsRead",  # Registros lidos durante shuffle
    "shuffleRecordsWritten",  # Registros escritos durante shuffle
    # Métricas de entrada/saída
    "recordsRead",  # Registros lidos
    "bytesRead",  # Bytes lidos
    "recordsWritten",  # Registros escritos
    "bytesWritten",  # Bytes escritos
    # Paralelismo
    "numStages",  # Número de estágios
    "numTasks",  # Número de tarefas
    "avg_active_tasks",  # Média de tarefas ativas (paralelismo)
]

In [23]:
# Função para criar gráficos para cada métrica
def create_metric_chart(metric_name, df):
    # Definir unidades e títulos baseados na métrica
    if 'Time' in metric_name or metric_name == 'elapsedTime':
        unit = 'Tempo (ms)'
    elif 'Bytes' in metric_name or 'Memory' in metric_name:
        unit = 'Bytes'
    elif 'Records' in metric_name:
        unit = 'Registros'
    else:
        unit = 'Valor'
    
    title = f'Comparação de {metric_name} com e sem AQE'
    
    # Criar o gráfico de barras
    fig = px.bar(
        df, 
        x='execution_type', 
        y=metric_name,
        title=title,
        labels={'value': unit, 'execution_type': 'Tipo de Execução'},
        text_auto=True,
        color='execution_type',
        color_discrete_map={'Com AQE': 'blue', 'Sem AQE': 'red'}
    )
    
    # Melhorar o layout
    fig.update_layout(
        xaxis_title="",
        yaxis_title=unit,
        legend_title="Execução"
    )
    
    return fig

In [24]:
# Criar e exibir gráficos para cada métrica
for metric in metricas_relevantes:
    fig = create_metric_chart(metric, df_combined)
    fig.show()

In [25]:
# Criar um gráfico de barras agrupadas para métricas de tempo
time_metrics = ["elapsedTime", "executorRunTime", "executorCpuTime", "jvmGCTime"]
fig_time = px.bar(
    df_combined.melt(
        id_vars=['execution_type', 'query'],
        value_vars=time_metrics,
        var_name='metric',
        value_name='value'
    ),
    x='metric',
    y='value',
    color='execution_type',
    barmode='group',
    title='Comparação de Métricas de Tempo',
    labels={'value': 'Tempo (ms)', 'metric': 'Métrica', 'execution_type': 'Tipo de Execução'}
)
fig_time.show()

In [26]:
# Criar um gráfico de barras para métricas de paralelismo
parallelism_metrics = ["numStages", "numTasks", "avg_active_tasks"]
fig_parallelism = px.bar(
    df_combined.melt(
        id_vars=['execution_type', 'query'],
        value_vars=parallelism_metrics,
        var_name='metric',
        value_name='value'
    ),
    x='metric',
    y='value',
    color='execution_type',
    barmode='group',
    title='Comparação de Métricas de Paralelismo',
    labels={'value': 'Quantidade', 'metric': 'Métrica', 'execution_type': 'Tipo de Execução'}
)
fig_parallelism.show()

In [27]:
# Criar um gráfico de barras para métricas de tempo de shuffle
shuffle_time_metrics = ["shuffleFetchWaitTime", "shuffleWriteTime"]
fig_shuffle_time = px.bar(
    df_combined.melt(
        id_vars=['execution_type', 'query'],
        value_vars=shuffle_time_metrics,
        var_name='metric',
        value_name='value'
    ),
    x='metric',
    y='value',
    color='execution_type',
    barmode='group',
    title='Comparação de Métricas de Tempo de Shuffle',
    labels={'value': 'Tempo (ms)', 'metric': 'Métrica', 'execution_type': 'Tipo de Execução'}
)
fig_shuffle_time.show()

In [28]:
# Criar um gráfico de barras para métricas de dados de shuffle
shuffle_data_metrics = ["shuffleTotalBytesRead", "shuffleBytesWritten", "shuffleRecordsRead", "shuffleRecordsWritten"]
fig_shuffle_data = px.bar(
    df_combined.melt(
        id_vars=['execution_type', 'query'],
        value_vars=shuffle_data_metrics,
        var_name='metric',
        value_name='value'
    ),
    x='metric',
    y='value',
    color='execution_type',
    barmode='group',
    title='Comparação de Métricas de Dados de Shuffle',
    labels={'value': 'Quantidade', 'metric': 'Métrica', 'execution_type': 'Tipo de Execução'}
)
fig_shuffle_data.update_layout(yaxis_type="log")  # Escala logarítmica para melhor visualização
fig_shuffle_data.show()

In [16]:
# Base path
base_path = '/home/guilhermerc/Documentos/workspace/pyspark-playground/output'

# Encontrar todas as pastas de queries
query_folders = [f for f in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, f)) and f.startswith('q')]

# Lista para armazenar todos os dataframes
all_dfs = []

# Para cada query, carregar todos os modos de execução disponíveis
for query in query_folders:
    query_path = os.path.join(base_path, query)
    execution_modes = [f for f in os.listdir(query_path) if os.path.isdir(os.path.join(query_path, f))]
    
    for mode in execution_modes:
        mode_path = os.path.join(query_path, mode)
        grouped_file = os.path.join(mode_path, f"{mode}.grouped")
        
        if os.path.exists(grouped_file):
            # Carregar o arquivo
            df = pd.read_csv(grouped_file)
            
            # Adicionar colunas para identificação
            df['execution_mode'] = mode
            df['query'] = query
            
            # Salvar na lista
            all_dfs.append(df)

# Combinar todos os dataframes
df_all_queries = pd.concat(all_dfs, ignore_index=True)

# Mapeamento para nomes mais amigáveis dos modos de execução
execution_mode_mapping = {
    'aqe': 'Com AQE',
    'wo': 'Sem otimizações',
    'cbo': 'Com CBO',
    'cboaqe': 'Com CBO+AQE'
}

# Aplicar o mapeamento
df_all_queries['execution_type'] = df_all_queries['execution_mode'].map(execution_mode_mapping)


In [17]:
# Gráfico de uso de memória
fig_memory = px.bar(
    df_all_queries,
    x='query',
    y='peakExecutionMemory',
    color='execution_type',
    barmode='group',
    title='Uso de Memória por Query e Tipo de Execução',
    labels={'peakExecutionMemory': 'Memória Máxima Utilizada', 'query': 'Query', 'execution_type': 'Tipo de Execução'}
)
fig_memory.update_layout(xaxis={'categoryorder': 'category ascending'})
fig_memory.show()

In [18]:
fig_execution_time = px.bar(
    df_all_queries,
    x='query',
    y='elapsedTime',
    color='execution_type',
    barmode='group',
    title='Tempo de Execução por Query e Tipo de Execução',
    labels={'elapsedTime': 'Tempo de Execução (ms)', 'query': 'Query', 'execution_type': 'Tipo de Execução'}
)
fig_execution_time.update_layout(xaxis={'categoryorder': 'category ascending'})
fig_execution_time.show()

In [19]:
# Gráfico de operações de shuffle
fig_shuffle = px.bar(
    df_all_queries,
    x='query',
    y=['shuffleBytesWritten', 'shuffleTotalBytesRead'],
    color='execution_type',
    barmode='group',
    title='Operações de Shuffle por Query e Tipo de Execução',
    labels={'value': 'Bytes', 'query': 'Query', 'execution_type': 'Tipo de Execução', 'variable': 'Operação'}
)
fig_shuffle.update_layout(xaxis={'categoryorder': 'category ascending'})
fig_shuffle.show()

In [20]:
# Calcular o speedup
pivot_df = df_all_queries.pivot_table(
    index=['query'], 
    columns='execution_mode', 
    values='elapsedTime'
).reset_index()

# Adicionar colunas de speedup
if 'wo' in pivot_df.columns:
    if 'aqe' in pivot_df.columns:
        pivot_df['speedup_aqe'] = pivot_df['wo'] / pivot_df['aqe']
    if 'cbo' in pivot_df.columns:
        pivot_df['speedup_cbo'] = pivot_df['wo'] / pivot_df['cbo']
    if 'cboaqe' in pivot_df.columns:
        pivot_df['speedup_cboaqe'] = pivot_df['wo'] / pivot_df['cboaqe']

# Transformar para formato long para plotagem
speedup_cols = [col for col in pivot_df.columns if col.startswith('speedup_')]
if speedup_cols:
    speedup_df = pivot_df.melt(
        id_vars=['query'],
        value_vars=speedup_cols,
        var_name='optimization',
        value_name='speedup'
    )
    
    # Melhorar rótulos para o gráfico
    speedup_df['optimization'] = speedup_df['optimization'].str.replace('speedup_', '')
    speedup_df['optimization'] = speedup_df['optimization'].map({
        'aqe': 'AQE',
        'cbo': 'CBO',
        'cboaqe': 'CBO+AQE'
    })
    
    # Plotar o speedup
    fig_speedup = px.bar(
        speedup_df,
        x='query',
        y='speedup',
        color='optimization',
        barmode='group',
        title='Speedup em Relação à Execução Sem Otimizações',
        labels={'speedup': 'Speedup (x vezes)', 'query': 'Query', 'optimization': 'Otimização'}
    )
    fig_speedup.update_layout(xaxis={'categoryorder': 'category ascending'})
    fig_speedup.add_hline(y=1, line_dash="dash", line_color="gray")
    fig_speedup.show()

In [21]:
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler

# Selecionar as métricas para o gráfico de radar
radar_metrics = ['elapsedTime', 'executorRunTime', 'executorCpuTime', 
                 'shuffleTotalBytesRead', 'shuffleBytesWritten', 'peakExecutionMemory']

# Para cada query, criar um gráfico de radar
for query in df_all_queries['query'].unique():
    query_data = df_all_queries[df_all_queries['query'] == query]
    
    # Normalizar os dados para a escala 0-1
    scaler = MinMaxScaler()
    query_data_norm = pd.DataFrame(
        scaler.fit_transform(query_data[radar_metrics]), 
        columns=radar_metrics
    )
    query_data_norm['execution_type'] = query_data['execution_type'].values
    
    # Criar gráfico de radar
    fig = go.Figure()
    
    for exec_type in query_data_norm['execution_type'].unique():
        exec_data = query_data_norm[query_data_norm['execution_type'] == exec_type]
        
        fig.add_trace(go.Scatterpolar(
            r=exec_data[radar_metrics].values[0],
            theta=radar_metrics,
            fill='toself',
            name=exec_type
        ))
    
    fig.update_layout(
        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[0, 1]
            )
        ),
        title=f'Comparação de Métricas para Query {query}',
        showlegend=True
    )
    fig.show()